In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import pandas as pd
import os
import sys
import datetime

root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    
load_dotenv()

True

In [2]:
import tensorflow as tf
tf.__version__, tf.config.list_physical_devices('GPU')

('2.10.1', [])

In [3]:
from training.utils import date_range, df_to_X_y

df_train = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2019, 11, 1), 
        datetime.date(2020, 1, 15),
        datetime.timedelta(days=1),
    )
])
df_val = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2020, 1, 15), 
        datetime.date(2020, 2, 15),
        datetime.timedelta(days=1),
    )
])

X_train, y_train = df_to_X_y(df_train)
X_val, y_val = df_to_X_y(df_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((1494497, 124), (1494497, 4), (683009, 124), (683009, 4))

In [4]:
from keras.models import Sequential, clone_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam


model = Sequential()
model.add(InputLayer((124, 1)))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(8, 'relu'))
model.add(Dense(1, 'linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 122, 64)           256       
                                                                 
 flatten (Flatten)           (None, 7808)              0         
                                                                 
 dense (Dense)               (None, 8)                 62472     
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 62,737
Trainable params: 62,737
Non-trainable params: 0
_________________________________________________________________


In [5]:
model_view = clone_model(model)
cp_view = ModelCheckpoint('../model/d-view.keras', save_best_only=True)
model_view.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_view.fit(X_train, y_train[:, 0], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_view])
model_view.save('../model/saved/d-view.keras')


Epoch 1/10
46704/46704 [==============================] - 175s 4ms/step - loss: 19.3391 - root_mean_squared_error: 4.3976 - val_loss: 80.7372 - val_root_mean_squared_error: 8.9854
Epoch 2/10
46704/46704 [==============================] - 161s 3ms/step - loss: 15.4077 - root_mean_squared_error: 3.9253 - val_loss: 74.4786 - val_root_mean_squared_error: 8.6301
Epoch 3/10
46704/46704 [==============================] - 161s 3ms/step - loss: 14.5758 - root_mean_squared_error: 3.8178 - val_loss: 68.8635 - val_root_mean_squared_error: 8.2984
Epoch 4/10
46704/46704 [==============================] - 157s 3ms/step - loss: 14.3259 - root_mean_squared_error: 3.7850 - val_loss: 86.2318 - val_root_mean_squared_error: 9.2861
Epoch 5/10
46704/46704 [==============================] - 170s 4ms/step - loss: 13.9673 - root_mean_squared_error: 3.7373 - val_loss: 83.4278 - val_root_mean_squared_error: 9.1339
Epoch 6/10
46704/46704 [==============================] - 177s 4ms/step - loss: 13.8094 - root_mean_

In [6]:

model_cart = clone_model(model)
cp_cart = ModelCheckpoint('../model/d-cart.keras', save_best_only=True)
model_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_cart.fit(X_train, y_train[:, 1], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_cart])
model_cart.save('../model/saved/d-cart.keras')


Epoch 1/10
46704/46704 [==============================] - 190s 4ms/step - loss: 8.6568 - root_mean_squared_error: 2.9422 - val_loss: 37.2984 - val_root_mean_squared_error: 6.1072
Epoch 2/10
46704/46704 [==============================] - 164s 4ms/step - loss: 7.9887 - root_mean_squared_error: 2.8264 - val_loss: 38.6631 - val_root_mean_squared_error: 6.2180
Epoch 3/10
46704/46704 [==============================] - 158s 3ms/step - loss: 7.6732 - root_mean_squared_error: 2.7700 - val_loss: 37.4212 - val_root_mean_squared_error: 6.1173
Epoch 4/10
46704/46704 [==============================] - 184s 4ms/step - loss: 7.5202 - root_mean_squared_error: 2.7423 - val_loss: 39.3478 - val_root_mean_squared_error: 6.2728
Epoch 5/10
46704/46704 [==============================] - 165s 4ms/step - loss: 7.4978 - root_mean_squared_error: 2.7382 - val_loss: 39.1054 - val_root_mean_squared_error: 6.2534
Epoch 6/10
46704/46704 [==============================] - 161s 3ms/step - loss: 7.3321 - root_mean_square

In [7]:

model_remove_from_cart = clone_model(model)
cp_remove_from_cart = ModelCheckpoint('../model/d-remove_from_cart.keras', save_best_only=True)
model_remove_from_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_remove_from_cart.fit(X_train, y_train[:, 2], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_remove_from_cart])
model_remove_from_cart.save('../model/saved/d-remove_from_cart.keras')


Epoch 1/10
46704/46704 [==============================] - 165s 4ms/step - loss: 7.8477 - root_mean_squared_error: 2.8014 - val_loss: 36.8185 - val_root_mean_squared_error: 6.0678
Epoch 2/10
46704/46704 [==============================] - 177s 4ms/step - loss: 3.3842 - root_mean_squared_error: 1.8396 - val_loss: 38.2349 - val_root_mean_squared_error: 6.1834
Epoch 3/10
46704/46704 [==============================] - 164s 4ms/step - loss: 3.3136 - root_mean_squared_error: 1.8203 - val_loss: 37.8864 - val_root_mean_squared_error: 6.1552
Epoch 4/10
46704/46704 [==============================] - 163s 3ms/step - loss: 3.2586 - root_mean_squared_error: 1.8052 - val_loss: 36.6992 - val_root_mean_squared_error: 6.0580
Epoch 5/10
46704/46704 [==============================] - 175s 4ms/step - loss: 3.2142 - root_mean_squared_error: 1.7928 - val_loss: 36.2559 - val_root_mean_squared_error: 6.0213
Epoch 6/10
46704/46704 [==============================] - 176s 4ms/step - loss: 3.2062 - root_mean_square

In [8]:

model_purchase = clone_model(model)
cp_purchase = ModelCheckpoint('../model/d-purchase.keras', save_best_only=True)
model_purchase.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_purchase.fit(X_train, y_train[:, 3], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_purchase])
model_purchase.save('../model/saved/d-purchase.keras')


Epoch 1/10
46704/46704 [==============================] - 151s 3ms/step - loss: 0.9329 - root_mean_squared_error: 0.9659 - val_loss: 41.9301 - val_root_mean_squared_error: 6.4753
Epoch 2/10
46704/46704 [==============================] - 144s 3ms/step - loss: 0.7133 - root_mean_squared_error: 0.8446 - val_loss: 41.3955 - val_root_mean_squared_error: 6.4339
Epoch 3/10
46704/46704 [==============================] - 141s 3ms/step - loss: 0.6988 - root_mean_squared_error: 0.8359 - val_loss: 42.6344 - val_root_mean_squared_error: 6.5295
Epoch 4/10
46704/46704 [==============================] - 144s 3ms/step - loss: 0.6907 - root_mean_squared_error: 0.8311 - val_loss: 42.5209 - val_root_mean_squared_error: 6.5208
Epoch 5/10
46704/46704 [==============================] - 143s 3ms/step - loss: 0.6841 - root_mean_squared_error: 0.8271 - val_loss: 44.6612 - val_root_mean_squared_error: 6.6829
Epoch 6/10
46704/46704 [==============================] - 141s 3ms/step - loss: 0.6833 - root_mean_square